In [ ]:
import xml.etree.ElementTree as ET
from os import listdir
import numpy as np
import pandas as pd
from flair.data import Sentence
import flair
import re
from tqdm import tqdm
from difflib import SequenceMatcher
import pickle

# Split text and tag

Smoker and Family_hist can have no tag

In [ ]:
file_path = "C:/Users/Leste/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
names = [f for f in listdir(file_path) if f.endswith('.xml')]
train = pd.DataFrame(np.zeros((len(names), 2), dtype=object), columns=['text', 'annotation'])
n = 0
for name in names:
    tree = ET.parse(file_path + name)
    root = tree.getroot()

    ## Get the text
    nt = re.sub('\n',' ',root[0].text)
    nt = re.sub('\t',' ',nt) 
    nt = re.sub('"',"'",nt)
    ## sample 214 has a weird character
    #nt = re.sub('>','&gt;',nt) 
    #nt = re.sub('<','&lt;',nt)
    train['text'][n] = nt
    n+=1

In [ ]:
x = 'hi    i love you'
re.sub('  +',' ',x)

In [ ]:
n = 0
for name in names:
    tree = ET.parse(file_path + name)
    root = tree.getroot()
    ## Get the labels
    # skip FAMILY_HIST: doesn't have a text
    #skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
    PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
    # get all labels except for PHI
    tag_list = []
    for k in range(len(root[1])):
        for m in range(len(root[1][k])):
            if root[1][k][m].attrib.keys().__contains__('text') == False:
                continue
            tag_list.append((root[1][k][m].attrib['text'],root[1][k][m].tag))
    # get PHI labels
    for k in range(PHI,len(root[1])):
        tag_list.append((root[1][k].attrib['text'],root[1][k].tag))
    train['annotation'][n] = tag_list
    n+=1

# Functions to convert dataframe into input format
## use wisely, very slow (520 files takes 40 min to create labels)
 Please use create_labs function instead of mark_sentences


In [ ]:
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string


def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'
        
    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

## replace !!mark_sentence!! to better label the text with more than one word
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)
            
    return text

def create_data(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            #text_ = text    
            print(train.index[train['text']== text].tolist())    
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text, match_list)

            #for i in d.keys():
            #    f.writelines(i + ' ' + d[i] +'\n')
            #f.writelines('\n')
            for i in range(d.shape[0]):
                f.writelines(d['word'][i] + ' ' + d['label'][i] +'\n')
            f.writelines('\n')
            
def main():
    ## An example dataframe.
    #data = pd.DataFrame([["Horses are too tall and they pretend to care about your feelings", [("Horses", "ANIMAL")]],
    #              ["Who is Shaka Khan?", [("Shaka Khan", "PERSON")]],
    #              ["I like London and Berlin.", [("London", "LOCATION"), ("Berlin", "LOCATION")]],
    #              ["There is a banyan tree in the courtyard", [("banyan tree", "TREE")]]], columns=['text', 'annotation'])
    data = train.copy()
    ## path to save the txt file.
    filepath = 'tttt.txt'
    ## creating the file.
    create_data(data, filepath)
    
if __name__ == '__main__':
    main()

# Bunch of code for testing errors
## trash

In [ ]:
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string
text = train['text'][0]
annotation = train['annotation'][0]
#pattern = pattern.strip()
match_list = []
for i in annotation:
    a, text_ = matcher(text, i[0])
    match_list.append((a[0][0], a[0][1], i[1]))


match_list

In [ ]:
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())
        index,num_words

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

x = create_labs(train['text'][0],match_list)

In [ ]:
x

## test how to find the word using index

In [ ]:
        
a = s.split()
a.index('smoking')
len(re.findall(r' +',s[0:start]))
a[86],len(s[start:end].split())
#temp_str = pd.DataFrame(s.split(), columns=['word'])
#temp_str['len'] = temp_str['word'].apply(lambda x: len(x))
#temp_str['cumsum'] = temp_str['len'].cumsum()
#temp_str

## find which label went crazy and can't match text

In [ ]:
match_list = []
text = train['text'][176]
#text = "Horses are too tall and they pretend to care about your feelings"
annotation = train['annotation'][176]
for i in annotation:
    print(i)
    a, text_ = matcher(text, i[0])
    print(a)
    match_list.append((a[0][0], a[0][1], i[1]))
    


In [ ]:
train['text'][214]
#tree = ET.parse(file_path + names[62])
#root = tree.getroot()
#root[0].text

In [ ]:
names[214]

In [ ]:
tree = ET.parse(file_path + names[0])
root = tree.getroot()
## Get the labels
# skip FAMILY_HIST: doesn't have a text
#skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
# get all labels except for PHI
tag_list = []
root[1][5][0].attrib['text']

# create a list of tag as tuple

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = 'C:/Users/Leste/Desktop/BDD data'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt')

In [ ]:
corpus.train[0].to_tagged_string('ner')

In [ ]:
file_path = "C:/Users/Leste/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
names = [f for f in listdir(file_path) if f.endswith('.xml')]
tree = ET.parse(file_path + names[0])
root = tree.getroot()

## skip FAMILY_HIST: doesn't have a text
skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')

## get all labels except for PHI
tag_list = []
for n in range(len(root[1])):
    if n == skip:
        continue
    for m in range(len(root[1][n])):
        tag_list.append((root[1][n][m].attrib['text'],root[1][n][m].tag))

## get PHI labels
for n in range(PHI,len(root[1])):
    tag_list.append((root[1][n].attrib['text'],root[1][n].tag))

## clean the text

In [ ]:
import re

file_path = "C:/Users/Leste/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
names = [f for f in listdir(file_path) if f.endswith('.xml')]
tree = ET.parse(file_path + names[0])
root = tree.getroot()
nt = re.sub('\n','',root[0].text)



## Use trained model from flair

In [ ]:
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter
splitter = SegtokSentenceSplitter()
tagger = SequenceTagger.load('ner')
test = text_list[names[0]]
sentences = splitter.split(test)
sentences


## Try training NER with BERT

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.datasets import UD_ENGLISH
# 1. get the corpus
corpus = UD_ENGLISH().downsample(0.1)


In [ ]:
import pandas as pd
print(pd.DataFrame(corpus.test[0]))

In [ ]:

# 2. what label do we want to predict?
label_type = 'upos'


# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)


In [ ]:

# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )


In [ ]:

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )


In [ ]:

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)


In [ ]:

# 7. run fine-tuning
trainer.fine_tune('resources/taggers/example-roberta',
                  learning_rate=0.1,
                  mini_batch_size=32,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  max_epochs = 10)